# gu_toolkit Notebook Test Bench

This notebook contains:
1. **Basic functionality checks** that can run in script-like mode.
2. **GUI-dependent checks** that specifically validate notebook widget behavior.

Run top-to-bottom for a full smoke test after major changes.


In [ ]:
# Setup
import sys
from pathlib import Path

try:
    _start = Path(__file__).resolve().parent
except NameError:
    _start = Path.cwd().resolve()

_pkg_root = _start
while _pkg_root != _pkg_root.parent and not (_pkg_root / "__init__.py").exists():
    _pkg_root = _pkg_root.parent
sys.path.insert(0, str(_pkg_root.parent))

from gu_toolkit import *

## Part A — Basic functionality (non-GUI core checks)

These tests focus on callable behavior, symbolic parsing, and figure bookkeeping.


In [ ]:
# A1: numpify on a symbolic expression
expr = parse_latex(r"x^2 + a x")
f = numpify(expr, vars=["x", "a"])
value = f(3.0, 2.0)
assert abs(value - 15.0) < 1e-9, f"A1 failed: value={value}"
value = f(2.0, 3.0)
assert abs(value - 10.0) < 1e-9, f"A1 failed: value={value}"

print("A1 passed.")

In [ ]:
# A2: freeze/unfreeze behavior
frozen = f.freeze(a=4.0)
assert abs(frozen(3.0) - 21.0) < 1e-9, "A2 freeze failed"

unfrozen = frozen.unfreeze("a")
assert abs(unfrozen(3.0, 2.0) - 15.0) < 1e-9, "A2 unfreeze failed"
print("A2 passed.")

In [ ]:
# A3: basic figure + plot registration
x, a = sp.symbols("x a")
fig = Figure(x_range=(-5, 5), y_range=(-3, 3), sampling_points=300)
with fig:
    plot(a * sp.sin(x), x, id="test-wave")
    parameter(a, value=1.0, min=0.0, max=2.0, step=0.1)
    current_params = params.snapshot()

assert "test-wave" in fig.plots, f"A3 plot missing; plots={list(fig.plots.keys())}"
assert a in current_params, (
    f"A3 parameter missing; params={list(current_params.keys())}"
)
print("A3 passed.")

## Part B — Notebook GUI-dependent checks

These checks rely on notebook rendering and interactive widgets, so they are difficult to validate in plain unit tests.

> For each test below: run the cell, interact with the UI, and then mark the check as pass/fail.


In [ ]:
# B1: Slider widgets render and drive trace updates
x, amp, freq = sp.symbols("x amp freq")
fig_gui = Figure(x_range=(-8, 8), y_range=(-2, 2), sampling_points=500)
with fig_gui:
    plot(amp * sp.sin(freq * x), x, id="gui-wave")
    parameter(amp, value=1.0, min=0.0, max=2.0, step=0.1)
    parameter(freq, value=1.0, min=0.2, max=4.0, step=0.1)
display(fig_gui)

print(
    "Manual check B1: move 'amp' and 'freq' sliders and verify the plot updates immediately."
)

In [ ]:
# B2: Slider state synchronization in notebook
with fig_gui:
    print("Initial params:", params.snapshot())
print(
    "Manual check B2: adjust sliders, rerun this cell, verify params values changed accordingly."
)

In [ ]:
# B3: Multiple renders do not duplicate stale widgets or traces
with fig_gui:
    show()
    show()
print(
    "Manual check B3: repeated render() should keep UI usable and avoid obvious duplicate-control artifacts."
)
# BUG
# ---------------------------------------------------------------------------
# NameError                                 Traceback (most recent call last)
# Cell In[18], line 3
#      1 # B3: Multiple renders do not duplicate stale widgets or traces
#      2 with fig_gui:
# ----> 3     show()
#      4     show()
#      5 print("Manual check B3: repeated render() should keep UI usable and avoid obvious duplicate-control artifacts.")
#
# NameError: name 'show' is not defined
#
# Show should be a method of figure and a context-managed function that displays the figure.
# Do not use Render as it does something different.

## Part C — Results summary

Use this cell for an at-a-glance summary of automated checks from Part A.


In [ ]:
print("\n=== Automated checks summary ===")
print("A1/A2/A3 use assert and will stop on first failure.")
print("Operator action required: visually inspect B1/B2/B3 outputs and UI behavior.")

## Optional sign-off template

- Environment: `<python version>`, `<jupyter/lab version>`
- Date:
- Reviewer:
- Automated checks passed: Yes/No
- GUI checks passed (B1/B2/B3): Yes/No
- Notes:


## Part D — Project-019 phase 5/6 checks (multi-view API + snapshot/codegen)

These checks validate the new tabbed multi-view interface functionality exposed to notebook users.


In [ ]:
# D1: Multi-view targeting, scoped info visibility, and context-managed view switch
x, a = sp.symbols("x a")
fig_p019 = Figure(x_range=(-5, 5), y_range=(-3, 3), sampling_points=300)
fig_p019.add_view(
    "alt", x_range=(-2, 2), y_range=(-1, 1), x_label="x_alt", y_label="y_alt"
)

with fig_p019:
    parameter(a, value=1.0, min=0.0, max=2.0, step=0.1)
    plot(a * sp.sin(x), x, id="p019-wave", view=("main", "alt"))
    info("shared", id="shared")
    info("alt-only", id="alt-only", view="alt")

assert fig_p019.active_view_id == "main", (
    "D1 failed: default active view should be main"
)
assert fig_p019.plots["p019-wave"].views == ("alt", "main"), (
    "D1 failed: plot memberships not captured"
)
assert fig_p019.info_output["shared"].layout.display == "block", (
    "D1 failed: shared card should be visible"
)
assert fig_p019.info_output["alt-only"].layout.display == "none", (
    "D1 failed: alt-only card should be hidden in main"
)

with fig_p019.view("alt"):
    assert fig_p019.active_view_id == "alt", (
        "D1 failed: context manager did not switch view"
    )
    assert fig_p019.info_output["alt-only"].layout.display == "block", (
        "D1 failed: alt-only card should be visible in alt"
    )

assert fig_p019.active_view_id == "main", (
    "D1 failed: context manager did not restore original view"
)
print("D1 passed.")

In [ ]:
# D2: Snapshot and codegen include multi-view topology + scoped metadata
snap = fig_p019.snapshot()
assert snap.active_view_id == "main", "D2 failed: active view id mismatch"
assert tuple(v.id for v in snap.views) == ("main", "alt"), (
    "D2 failed: snapshot views mismatch"
)
assert snap.plots["p019-wave"].views == ("alt", "main"), (
    "D2 failed: plot views missing from snapshot"
)
assert any(
    card.id == "alt-only" and card.view_id == "alt" for card in snap.info_cards
), "D2 failed: scoped info card missing in snapshot"

code = fig_p019.to_code()
assert "fig.add_view('alt'" in code, "D2 failed: codegen missing add_view"
assert "view=('alt', 'main')" in code or "view=('main', 'alt')" in code, (
    "D2 failed: codegen missing plot view membership"
)
assert "info('alt-only', id='alt-only', view='alt')" in code, (
    "D2 failed: codegen missing scoped info"
)
print("D2 passed.")

In [ ]:
print("Project-019 notebook checks passed: D1/D2")
print(
    "Operator action (optional): display(fig_p019) and manually switch tabs to confirm UX behavior."
)